In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import wecorules as weco
from datetime import datetime

In [3]:
data = pd.read_csv("C:\\Users\\Sampreeth Salveru\\Downloads\\AG_Discrete_DS06-Jul-2020.csv")
excel = pd.read_excel("C:\\Users\\Sampreeth Salveru\\Downloads\\riskQA.xlsx",index_col="Quality Attributes")
data['Date'] = pd.to_datetime(data['PSDate'])
data['Date'] = data.Date.dt.strftime('%Y-%m-%d')
filter_col = [col for col in data if col.startswith(('Drug','Load'))]

In [4]:
for i in range(len(np.array_split(data['DrugSubstance_Sodium'].dropna(), len(data['DrugSubstance_Sodium'].dropna()) // 30))):
    x = np.array(np.array_split(data['DrugSubstance_Sodium'].dropna(), len(data['DrugSubstance_Sodium'].dropna()) // 30)[i])
    search = np.array(np.array_split(data['DrugSubstance_Sodium'].dropna(), len(data['DrugSubstance_Sodium'].dropna()) // 30)[i].index)[0]
    search1 = np.array(np.array_split(data['DrugSubstance_Sodium'].dropna(), len(data['DrugSubstance_Sodium'].dropna()) // 30)[i].index)[-1]
    beg = data.Date[search]
    end = data.Date[search1]
    mn = np.mean(x)
    std = np.std(x)
    leng = len(x)
#     print(mn,std)
#     print(leng, mn, std)
    print(beg+" | "+end)

2015-04-10 | 2015-10-21
2015-10-24 | 2017-03-24
2017-03-26 | 2018-05-02
2018-05-05 | 2019-04-13
2019-04-16 | 2019-12-21


In [5]:
unwanted = {'DrugSubstance_Osmolality','DrugSubstance_Copper','DrugSubstance_DNA','DrugSubstance_MicrobialLoad','DrugSubstance_SECFluorescenceReg1','DrugSubstance_MouseIgG','DrugSubstance_OverallRecovery'
           ,'DrugSubstance_SECFluorescenceReg2','DrugSubstance_ThrombinPepPeak2Area_','Load_LoadDensity'}
item_list = [e for e in filter_col if e not in unwanted]

In [6]:

def cpkppk(col):
    mean = excel.loc[col,"Mean"]
    ucl = excel.loc[col,"UCL"]
    lcl = excel.loc[col,"LCL"]
    usl = excel.loc[col,"USL"]
    lsl = excel.loc[col,"LSL"]
    units = excel.loc[col,"Units"]
    count1 = len(weco.weco1(col, ucl, lcl))
    count4 = len(weco.weco4(col, mean))
    count5 = len(weco.weco5(col))
    total_count = count1 + count4 + count5
    if total_count > 0:
        percent1 = count1/total_count
    else:
        percent1 = 0
    work = np.array([])
    work1 = np.array([])
    begdate = np.array([])
    enddate = np.array([])


    for i in range(len(np.array_split(data[col].dropna(), len(data[col].dropna()) // 30))):
        x = np.array(np.array_split(data[col].dropna(), len(data[col].dropna()) // 30)[i])
        search = np.array(np.array_split(data[col].dropna(), len(data[col].dropna()) // 30)[i].index)[0]
        search1 = np.array(np.array_split(data[col].dropna(), len(data[col].dropna()) // 30)[i].index)[-1]
        beg = data.Date[search]
        end = data.Date[search1]
        begdate = np.append(begdate, beg)
        enddate = np.append(enddate,end)
        mn = np.mean(x)
        std = np.std(x)
        
        if total_count == 0 and std > 0:
            if np.isnan(usl) == False and np.isnan(lsl) == False:
                cpkcalc = [(usl - mn)/(3*std), (mn-lsl)/(3*std)]
                cpk = min(cpkcalc)
                work = np.append(work, cpk)
                work1 = np.append(work1, "cpk")
            elif np.isnan(usl) == True:
                cpk = (mn-lsl)/(3*std)
                work = np.append(work, cpk)
                work1 = np.append(work1, "cpk")
            elif np.isnan(lsl) == True:
                cpk = (usl-mn)/(3*std)
                work = np.append(work, cpk)
                work1 = np.append(work1, "cpk")
        elif percent1 >= 0.02 and std > 0:
            if np.isnan(usl) == False and np.isnan(lsl) == False:
                ppkcalc = [(usl - mn)/(3*std), (mn-lsl)/(3*std)]
                ppk = min(ppkcalc)
                work = np.append(work, ppk)
                work1 = np.append(work1, "ppk")
            elif np.isnan(usl) == True:
                ppk = (mn - lsl)/(3*std)
                work = np.append(work, ppk)
                work1 = np.append(work1, "ppk")                
            elif np.isnan(lsl) == True:
                ppk = (usl - mn)/(3*std)
                work = np.append(work, ppk)
                work1 = np.append(work1, "ppk")                
        elif percent1 <= 0.02 and std > 0:
            if np.isnan(usl) == False and np.isnan(lsl) == False:
                cpkcalc = [(usl - mn)/(3*std), (mn-lsl)/(3*std)]
                cpk = min(cpkcalc)
                work = np.append(work, cpk)
                work1 = np.append(work1, "cpk")
            elif np.isnan(usl) == True:
                cpk = (mn-lsl)/(3*std)
                work = np.append(work, cpk)
                work1 = np.append(work1, "cpk")                
            elif np.isnan(lsl) == True:
                cpk = (usl-mn)/(3*std)
                work = np.append(work, cpk)
                work1 = np.append(work1, "cpk")                
    
    return [work,work1,begdate,enddate]
        

cpkppk(col = 'DrugSubstance_HCP')



[array([1.51490787, 1.95892426, 1.70706663, 2.76158183]),
 array(['ppk', 'ppk', 'ppk', 'ppk'], dtype='<U32'),
 array(['2013-01-17', '2015-08-20', '2017-01-28', '2018-02-28'],
       dtype='<U32'),
 array(['2015-08-15', '2017-01-25', '2018-02-24', '2018-07-25'],
       dtype='<U32')]

In [7]:
df = pd.DataFrame(columns = ["Parameter","Calculation","cpk/ppk","Date Range Beg","Date Range End"])

In [8]:
blah = np.array([])
blah1 = np.array([])
blah2 = np.array([])
blah3 = np.array([])
blah4 = np.array([])

for x in item_list:
    for i in range(len(cpkppk(x)[0])):
        print([x,cpkppk(x)[0][i],cpkppk(x)[1][i],cpkppk(x)[2][i],cpkppk(x)[3][i]])
        blah = np.append(blah, x)
        blah1 = np.append(blah1, cpkppk(x)[0][i])
        blah2 = np.append(blah2, cpkppk(x)[1][i])
        blah3 = np.append(blah3, cpkppk(x)[2][i])
        blah4 = np.append(blah4, cpkppk(x)[3][i])

['DrugSubstance_Calcium', 1.6321472306592462, 'ppk', '2013-01-17', '2015-08-05']
['DrugSubstance_Calcium', 1.6580040215706662, 'ppk', '2015-08-08', '2016-12-15']
['DrugSubstance_Calcium', 1.7116413281674425, 'ppk', '2016-12-16', '2017-06-28']
['DrugSubstance_Calcium', 2.3073239925660265, 'ppk', '2017-07-01', '2018-06-02']
['DrugSubstance_Calcium', 1.9688347499312087, 'ppk', '2018-06-06', '2019-04-20']
['DrugSubstance_Calcium', 2.411681401974198, 'ppk', '2019-04-23', '2019-12-21']
['DrugSubstance_Glycine', 1.0454037010101058, 'ppk', '2013-01-17', '2015-08-05']
['DrugSubstance_Glycine', 0.510356964638127, 'ppk', '2015-08-08', '2016-12-15']
['DrugSubstance_Glycine', 1.0401493891369993, 'ppk', '2016-12-16', '2017-06-28']
['DrugSubstance_Glycine', 1.3269776053940743, 'ppk', '2017-07-01', '2018-06-02']
['DrugSubstance_Glycine', 1.9917281772559898, 'ppk', '2018-06-06', '2019-04-23']
['DrugSubstance_Glycine', 1.5811388300841898, 'ppk', '2019-04-25', '2019-12-21']
['DrugSubstance_HCP', 1.514907

['DrugSubstance_SECFluorescenceReg4', 1.7868582950079948, 'ppk', '2019-04-25', '2019-12-21']
['DrugSubstance_Sodium', 0.6304843875768761, 'ppk', '2015-04-10', '2015-10-21']
['DrugSubstance_Sodium', 0.8693356158401709, 'ppk', '2015-10-24', '2017-03-24']
['DrugSubstance_Sodium', 0.7816798229585444, 'ppk', '2017-03-26', '2018-05-02']
['DrugSubstance_Sodium', 0.7961173386514133, 'ppk', '2018-05-05', '2019-04-13']
['DrugSubstance_Sodium', 0.97669316537764, 'ppk', '2019-04-16', '2019-12-21']
['DrugSubstance_SpecificActivity', 0.916964501641333, 'ppk', '2013-01-17', '2015-08-05']
['DrugSubstance_SpecificActivity', 1.8201641446478845, 'ppk', '2015-08-08', '2016-12-15']
['DrugSubstance_SpecificActivity', 1.1900988779500516, 'ppk', '2016-12-16', '2017-06-28']
['DrugSubstance_SpecificActivity', 1.4363285595963071, 'ppk', '2017-07-01', '2018-06-02']
['DrugSubstance_SpecificActivity', 0.91928328744244, 'ppk', '2018-06-06', '2019-04-23']
['DrugSubstance_SpecificActivity', 1.2333521743507443, 'ppk', 

In [9]:
dff = pd.DataFrame()
dff["Parameter"] = blah
dff["Calculation"]= blah1
dff["cpk/ppk"]= blah2
dff["Date Range Beg"]= blah3
dff["Date Range End"]= blah4

In [25]:

dff['Date'] = dff["Date Range Beg"] + " to " + dff["Date Range End"]
dff

,Parameter,Calculation,cpk/ppk,Date Range Beg,Date Range End,Date
0,DrugSubstance_Calcium,1.632147,ppk,2013-01-17,2015-08-05,2013-01-17 to 2015-08-05
1,DrugSubstance_Calcium,1.658004,ppk,2015-08-08,2016-12-15,2015-08-08 to 2016-12-15
2,DrugSubstance_Calcium,1.711641,ppk,2016-12-16,2017-06-28,2016-12-16 to 2017-06-28
3,DrugSubstance_Calcium,2.307324,ppk,2017-07-01,2018-06-02,2017-07-01 to 2018-06-02
4,DrugSubstance_Calcium,1.968835,ppk,2018-06-06,2019-04-20,2018-06-06 to 2019-04-20
...,...,...,...,...,...,...
116,DrugSubstance_TotalProtein,1.334445,ppk,2013-07-04,2016-11-02,2013-07-04 to 2016-11-02
117,DrugSubstance_TotalProtein,1.662675,ppk,2016-11-05,2017-05-31,2016-11-05 to 2017-05-31
118,DrugSubstance_TotalProtein,1.637067,ppk,2017-06-07,2018-05-09,2017-06-07 to 2018-05-09
119,DrugSubstance_TotalProtein,1.186570,ppk,2018-05-12,2019-04-16,2018-05-12 to 2019-04-16


In [27]:
joe = dff.loc[dff.Parameter=='DrugSubstance_Calcium']
joe

,Parameter,Calculation,cpk/ppk,Date Range Beg,Date Range End,Date
0,DrugSubstance_Calcium,1.632147,ppk,2013-01-17,2015-08-05,2013-01-17 to 2015-08-05
1,DrugSubstance_Calcium,1.658004,ppk,2015-08-08,2016-12-15,2015-08-08 to 2016-12-15
2,DrugSubstance_Calcium,1.711641,ppk,2016-12-16,2017-06-28,2016-12-16 to 2017-06-28
3,DrugSubstance_Calcium,2.307324,ppk,2017-07-01,2018-06-02,2017-07-01 to 2018-06-02
4,DrugSubstance_Calcium,1.968835,ppk,2018-06-06,2019-04-20,2018-06-06 to 2019-04-20
5,DrugSubstance_Calcium,2.411681,ppk,2019-04-23,2019-12-21,2019-04-23 to 2019-12-21


In [47]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(joe.Date),
                         y=list(joe.Calculation),
                         mode="markers",
                         name="cpk/ppk",
#                          hoverinfo ="text",
                         hovertext =joe["cpk/ppk"],
                         line = dict(color="black")
                         ))
fig.add_trace(go.Scatter(
                x=list(joe.Date),
                y=[1.33]*len(x),
                hoverinfo = "skip",
                mode="lines",
                line=dict(color="green",width=1.5)))
fig.add_trace(go.Scatter(
                x=list(joe.Date),
                y=[1]*len(x),
                hoverinfo = "skip",
                mode="lines",
                line=dict(color="green",width=1.5)))
fig.update_layout(showlegend=False,title_text="Kovaltry " +joe.Parameter[0]+"  " + datetime.today().strftime('%Y-%m-%d'),
                  plot_bgcolor='rgb(229,229,229)'
                 )
fig.show()